# Loan Application Orchestration Pipeline Demo (Agent Orchestration)

This notebook demonstrates the full loan application decision process **using the agent orchestration pipeline**. The orchestration function calls all tools and combines the results into a final summary.

In [1]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from utils.config_loader import ConfigLoader
from orchestrator_pipeline import orchestrate_application
from tools.synthesize_summary import synthesize_summary
from agents import RunContextWrapper

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import get_recommendation
from agents.orchestrator_agent import orchestrator_agent
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

#### Test example

In [2]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

## Integrate and orchestrate the agents

In [3]:
# Create a python function that will orchestrate the deterministic workflow
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))

print("Fraud Result:", fraud_result)
print("SLA Result:", sla_result)
print("Interest Rate Result:", interest_rate_result)

Fraud Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}
SLA Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}
Interest Rate Result: {'level': 'approve', 'label': 'Excellent'}


/Users/dariaz/Desktop/Oxford AI Summit - Full code summit assignment/Loan-application-agent/agents/interest_rate_agent.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


In [6]:
recommendation = get_recommendation(fraud_result, sla_result, interest_rate_result)
print("Recommendation:", recommendation)

Recommendation: **Recommendation: Reject the Loan Application**

The loan application should be rejected due to SLA violations in the KYC process, which exceeded configured limits. Although there are no fraud indicators detected and the interest rate is excellent, compliance with SLA is critical for maintaining operational integrity and risk management.


In [4]:
# Example: Use interest_rate_agent to check for interest rate (sync call for demo)
interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))
print('Interest Rate Agent Result:', interest_rate_result)


Interest Rate Agent Result: {'level': 'approve', 'label': 'Excellent'}


/Users/dariaz/Desktop/Oxford AI Summit - Full code summit assignment/Loan-application-agent/agents/interest_rate_agent.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


In [38]:
# Example: Use fraud_agent to check for fraud indicators (sync call for demo)
fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
print('Fraud Agent Result:', fraud_result)

Fraud Agent Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}


In [39]:
# Example: Use sla_agent to check SLA compliance (sync call for demo)
sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
print('SLA Agent Result:', sla_result)

SLA Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


In [41]:
# Example: Use orchestrator_agent to run the full orchestration (sync call for demo)
orc_result = orchestrator_agent.tools[0](RunContextWrapper(test_app))
print('Orchestrator Agent Result:', orc_result)

Orchestrator Agent Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}


# TO DO:

## 1.Trend Analysis

### ToDo: Ask agents to analyze and take decisions based on the metrics below: 
####- interest rate cutoff : search the web to find what a reasonable interest rate is for a loan of this type -> 
####- fraud detection: based on the kyc and other metrics 
#- SLA compliance

### Goal: give AI more freedom to decide BY REMOVING THE CONFIG 

## 2.MCP Tools 
### Send email with report 


In [8]:
# Prepare the context for the report agent
final_context = {
    "recommendation": getattr(recommendation, 'final_output', recommendation),
    "fraud_result": fraud_result,
    "sla_result": sla_result,
    "interest_rate_result": interest_rate_result
}

# Generate the summary report
report_result = report_agent.tools[0](RunContextWrapper(final_context))
print("System Summary Report:\n", report_result)

System Summary Report:
 📋 Loan Application System Summary
Dear Daria Zahaleanu,  
Recommendation: **Recommendation: Reject the Loan Application**

The loan application should be rejected due to SLA violations in the KYC process, which exceeded configured limits. Although there are no fraud indicators detected and the interest rate is excellent, compliance with SLA is critical for maintaining operational integrity and risk management.
Fraud Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}
SLA Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}
Interest Rate Result: {'level': 'approve', 'label': 'Excellent'}

Regards,
Team 5 - Loan Application Agent



In [9]:
from mcp.zappier_channel import ZapierChannel
import os

# Set your Zapier webhook URL (from your environment or hardcoded for demo)
zapier_url = os.getenv("MCP_SERVER_URL_SSE", "https://hooks.zapier.com/hooks/catch/23570793/u3m6t1v/")

zapier_channel = ZapierChannel(webhook_url=zapier_url)

# Send the report via Zapier
zapier_channel.send(
    message=report_result,
    metadata={"subject": "Loan Application Recommendation", "to": "anadaria.zahaleanu@gmail.com"}
)

print("✅ Report sent via Zapier email webhook.")

✅ Report sent via Zapier email webhook.
